In [1]:
# test

# Intro

team name

Mention limitations from the given data

report location/background/info: Prepare a report containing your results from the analysis. It should contain the following: Intro, data cleaning/pre-processing, visualizations (at least 3), analysis/results, conclusion/future improvements.

In [272]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import string
from collections import Counter
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import spacy
from spacy import displacy

import nltk

In [42]:
# Remove automatic formatting with symbol usage (Ex: $ sign -> MathJax)
pd.options.display.html.use_mathjax=False

In [261]:
# Read training data
train_data = pd.read_csv('../data/advanced_trainset.csv')
train_data.head()

,Sentence,Sentiment
0,According to the Finnish-Russian Chamber of Co...,neutral
1,The Swedish buyout firm has sold its remaining...,neutral
2,$SPY wouldn't be surprised to see a green close,positive
3,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative


In [270]:
train_json = train_data.rename({'Sentence': 'text', 'Sentiment': 'label'}, axis=1).to_json(orient='records', lines=True)

In [278]:
train_json

'{"text":"According to the Finnish-Russian Chamber of Commerce , all the major construction companies of Finland are operating in Russia .","label":"neutral"}\n{"text":"The Swedish buyout firm has sold its remaining 22.4 percent stake , almost eighteen months after taking the company public in Finland .","label":"neutral"}\n{"text":"$SPY wouldn\'t be surprised to see a green close","label":"positive"}\n{"text":"Shell\'s $70 Billion BG Deal Meets Shareholder Skepticism","label":"negative"}\n{"text":"SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANGE RELEASE OCTOBER 14 , 2008 AT 2:45 PM The Company updates its full year outlook and estimates its results to remain at loss for the full year .","label":"negative"}\n{"text":"The Stockmann department store will have a total floor space of over 8,000 square metres and Stockmann \'s investment in the project will have a price tag of about EUR 12 million .","label":"neutral"}\n{"text":"Circulation revenue has increased by 5 % in Finland and 4 % in 

In [284]:
with open('../data/train_data.jsonl', 'w') as outfile:
    outfile.write(train_json)

In [285]:
openai.File.create(
  file=open("../data/train_data.jsonl"),
  purpose='classifications'
)

<File file id=file-RAuC755dLssTr6hr8QW2yb1V at 0x1c9bf67b630> JSON: {
  "bytes": 648721,
  "created_at": 1649568996,
  "filename": "train_data.jsonl",
  "id": "file-RAuC755dLssTr6hr8QW2yb1V",
  "object": "file",
  "purpose": "classifications",
  "status": "uploaded",
  "status_details": null
}

In [292]:
file_id = openai.File.list()['data'][0]['id']

'file-RAuC755dLssTr6hr8QW2yb1V'

In [44]:
# Read testing data
test_data = pd.read_csv('../data/advanced_testset.csv')
test_data.head()

,Sentence
0,Earnings per share ( EPS ) dropped to EUR 0.21...
1,$SONC Amazing run since middle of March - obvi...
2,"Ruukki Romania , the local arm of Finnish meta..."
3,Self-service and automation are in a bigger ro...
4,Alma Media 's operating profit amounted to EUR...


In [48]:
# Read supplementary stock ticker data
stocks = pd.read_csv('../data/stock_tickers.csv')
stocks.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$134.87,-1.0600,-0.78%,4.047629e+10,United States,1999.0,2070939,Capital Goods,Electrical Products
1,AA,Alcoa Corporation Common Stock,$84.15,-1.9500,-2.265%,1.551901e+10,NaN,2016.0,4585478,Basic Industries,Metal Fabrications
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.83,0.0200,0.204%,1.228750e+09,NaN,2021.0,186747,Finance,Business Services
3,AACG,ATA Creativity Global American Depositary Shares,$1.21,-0.0600,-4.724%,3.796607e+07,China,NaN,7154,Miscellaneous,Service to the Health Industry
4,AACI,Armada Acquisition Corp. I Common Stock,$9.9781,0.1181,1.198%,2.066415e+08,United States,2021.0,174251,Consumer Durables,Consumer Electronics/Appliances


In [171]:
# Set spacy NLP English pipeline
nlp = spacy.load('en_core_web_sm')

# EDA

Look through the dataset for things that catch your eye. What proportion of responses are negative, positive, and neutral? Do you see any imbalances in the data? What else do you find? Please provide charts and visualizations to support your claim.

In [25]:
sentiment_counts = train_data['Sentiment'].value_counts().to_frame().reset_index()
sentiment_counts

,index,Sentiment
0,neutral,2363
1,positive,1383
2,negative,636


In [26]:
fig = px.bar(sentiment_counts, x='index', y='Sentiment', \
             title="Sentiment Counts in Training Data", labels={'index':'Sentiment', 'Sentiment': 'Count'})
fig.show()

As we see from this bar chart, there is a significant imbalance in the number of observations we have for neutral, positive, and negative sentences. This will mean... TODO: HERE

## Subject of Sentences

Another point of interest is to identify the subject of the sentence. This gives us an idea of what the sentiment is directed towards. For example, if the sentence is "AAPL is popping off," we would want to identify the sentiment as well as what the sentiment is directed towards. This process is a combination of EDA and feature engineering, so we will include visualizations here and the actual data manipulation in the **Feature Engineering** section.

TODO: talk about tokenization here

In [178]:
sent = a.loc[0]['Sentence']
doc=nlp(sent)
displacy.render(doc, style="dep")

In [179]:
displacy.render(doc, style="ent")

With this visualization, we can see the breakdown of the sentence and determine the subjects as well as the relations between different words. However, as we can see in this example, the spacy NLP processing is not quite able to identify complex sentence tokens such as the "Finnish-Russian Chamber of Commerce." Thus, we will have to select multiple groups to identify as subjects...

## Negative Sentences

In [106]:
# Convert Sentence series of negative sentiment into a string for EDA purposes
neg_words = train_data[train_data['Sentiment'] == 'negative']['Sentence'].str.cat(sep=' ')
neg_words = neg_words.split(' ')

In [107]:
# Grab as many words as possible while ignoring numbers, or incorrectly formatted words
neg_words = [word.strip().lower() for word in neg_words if not any(c for c in word.strip() if c not in string.ascii_letters + "'")]

In [109]:
Counter(neg_words).most_common()[:10]

[('the', 524),
 ('in', 346),
 ('of', 314),
 ('to', 287),
 ('eur', 228),
 ('a', 185),
 ('mn', 164),
 ('from', 151),
 ('and', 149),
 ('for', 125)]

As we can see from the top 10 most common words in negative sentiment, it's impossible to gauge distinct or important words that correlate with negative sentiment. In order to find the more important words, we can calculate the term frequency - inverse data frequency score for each word and identify highest weighted words.

TODO: discuss tf-idf formula and reasoning here.

In [115]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(neg_words)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [120]:
df

,aaland,ab,aberdeen,about,above,abp,acanb,acando,accommodation,according,...,year,years,yesterday,yet,yit,york,you,yoy,zoltan,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature Engineering

Do you need to make any changes to “Sentence” to make it more digestible for your model? Will you make any restrictions to your sample? Even if you don’t choose to make any changes to the data, please describe your reasoning.

We want to map the sentences to a specific stock, market, or even country. In order to do this, we imported the stock_tickers.csv file (renamed) from the NASDAQ website to try matching words in the sentences to a specific ticker. However, our data exploration revealed that some sentences didn't contain clean tickers or even any tickers at all, so we'll be attempting to create a category that contains information on the `subject` of the sentence.

To do this, we will use spacy, (TODO: describe spacy here!!)...

In [190]:
def get_subject(sent):
    '''
    Tokenizes and identifies the subject of the sentence using spacy's English pipeline.
    '''
    doc=nlp(sent)
    sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj" or tok.pos_ == "PROPN")]
    return sub_toks

In [191]:
a = train_data.copy()
a

,Sentence,Sentiment
0,According to the Finnish-Russian Chamber of Co...,neutral
1,The Swedish buyout firm has sold its remaining...,neutral
2,$SPY wouldn't be surprised to see a green close,positive
3,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
...,...,...
4377,Investments in product development stood at 6....,neutral
4378,HSBC Says Unit to Book $585 Million Charge on ...,negative
4379,RISING costs have forced packaging producer Hu...,negative
4380,"In the building and home improvement trade , s...",neutral


In [192]:
a['Subject'] = a['Sentence'].apply(get_subject)

In [193]:
a

,Sentence,Sentiment,Subject
0,According to the Finnish-Russian Chamber of Co...,neutral,"[Chamber, Commerce, companies, Finland, Russia]"
1,The Swedish buyout firm has sold its remaining...,neutral,"[firm, Finland]"
2,$SPY wouldn't be surprised to see a green close,positive,[SPY]
3,Shell's $70 Billion BG Deal Meets Shareholder ...,negative,"[Shell, BG, Shareholder, Skepticism]"
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative,"[SSH, COMMUNICATIONS, SECURITY, CORP, STOCK, E..."
...,...,...,...
4377,Investments in product development stood at 6....,neutral,[Investments]
4378,HSBC Says Unit to Book $585 Million Charge on ...,negative,"[HSBC, Unit, Book]"
4379,RISING costs have forced packaging producer Hu...,negative,"[RISING, costs, Huhtamaki, Hampshire]"
4380,"In the building and home improvement trade , s...",neutral,"[sales, EUR, mn]"


# Model Building

Create a NLP model that uses the “Sentence” as an input, using “Sentiment”  as labels. Ideally, you will compare the results of several different models to find the optimal choice. What led you to choose your final model? Did you run into any roadblocks? Please describe your process in depth. Make sure to train your model on the training set only.

In [195]:
# TESTING DIFFERENT MODELS TALK ABOUT IT

In [244]:
import yaml
import openai

In [230]:
with open('../config/config.yml', "r") as f:
    config = yaml.safe_load(f)

In [247]:
openai_key = config['open-ai-key']
openai.api_key = openai_key
openai_engines = ['ada', 'babbage', 'curie', 'davinci']

In [239]:
X_train, X_test, y_train, y_test = train_test_split(train_data['Sentence'], train_data['Sentiment'], test_size=0.20, random_state=69420)

In [258]:
openai_input = list(map(list, zip(X_train, y_train)))
openai_input

[['Cameco typically prices sales contracts using a 40:60 ratio of fixed prices and spot prices .',
  'neutral'],
 ['The total investment in 2006 and 2007 is expected to amount to about EUR75m .',
  'neutral'],
 ['Kaupthing Bank will publish its annual results for 2007 before markets open on Thursday 31 January .',
  'neutral'],
 ['Finnish diversified holding company Aspo Oyj said on September 8 , 2008 that it will sell the Finnish tape business of its Kaukomarkkinat unit to local Oy Telpak Ab .',
  'neutral'],
 ["A corresponding increase of 85,432.50 euros in Ahlstrom 's share capital has been entered in the Trade Register today .",
  'neutral'],
 ["Around 50 percent of the world 's ro-ro fleet is over 25 years old and needs to be scrapped for environmental reasons .",
  'neutral'],
 ['The EA Reng group posted sales of approximately 84 million kroons for 2007 .',
  'neutral'],
 ["The Finnish daily Kauppalehti surmises that Finnish supplier Rautaruukki has raised its prices above Aker '

In [259]:
openai.File.create(
  file=open("../data/advanced_trainset.csv"),
  purpose='classifications'
)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 216753: character maps to <undefined>

In [293]:
def classify_sentiment(sent):
    openai.Classification.create(
      search_model="ada",
      model="curie",
      file='file-RAuC755dLssTr6hr8QW2yb1V',
      query=sent,
      labels=["positive", "negative", "neutral"],
    )

    return response

In [294]:
preds = X_test.apply(classify_sentiment)

RateLimitError: you exceeded your current quota, please check your plan and billing details

In [295]:
preds

NameError: name 'preds' is not defined

In [216]:
dataset = train_data.copy()

In [227]:
dataset

,Sentence,Sentiment
0,According Finnish-Russian Chamber Commerce maj...,neutral
1,The Swedish buyout firm sold remaining 224 per...,neutral
2,SPY wouldnt surprised see green close,positive
3,Shells 70 Billion BG Deal Meets Shareholder Sk...,negative
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
...,...,...
4377,Investments product development stood 60 mln e...,neutral
4378,HSBC Says Unit Book 585 Million Charge Settlement,negative
4379,RISING costs forced packaging producer Huhtama...,negative
4380,In building home improvement trade sales decre...,neutral


## NLTK

In [217]:
undesired = string.punctuation.replace('-', '')
def punc_clean(text):
    a=[w for w in text if w not in undesired]
    return ''.join(a)
dataset['Sentence'] = dataset['Sentence'].apply(punc_clean)
dataset.head(2)

,Sentence,Sentiment
0,According to the Finnish-Russian Chamber of Co...,neutral
1,The Swedish buyout firm has sold its remaining...,neutral


In [218]:
# TODO: LOOK FOR WORDS DROPPED BY STOPWORDS THAT QE NDEED

def remove_stopwords(text):
    stopword = nltk.corpus.stopwords.words('english')
    stopword.remove('not')
    a=[w for w in nltk.word_tokenize(text) if w not in stopword]
    return ' '.join(a)
dataset['Sentence'] = dataset['Sentence'].apply(remove_stopwords)

In [220]:
vector = TfidfVectorizer(ngram_range=(1,2),min_df=1) # TODO: ADJUST PARAMS
vector.fit(dataset['Sentence'])
vect_X = vector.transform(dataset['Sentence'])

In [222]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
clf = model.fit(vect_X, dataset['Sentiment'])
clf.score(vect_X, dataset['Sentiment']) * 100 # TODO: CHANGE SCORE TO f1 SCORE

87.03788224554998

In [199]:
X_train, X_test, y_train, y_test = train_test_split(train_data['Sentence'], train_data['Sentiment'], test_size=0.20, random_state=69420)

## Tensorflow

In [223]:
import tensorflow as tf

In [224]:
X_train, X_test, y_train, y_test = train_test_split(train_data['Sentence'], train_data['Sentiment'], test_size=0.20, random_state=69420)

In [225]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

## Pytorch

## Sci-kit Learn

In [194]:
train_data

,Sentence,Sentiment
0,According to the Finnish-Russian Chamber of Co...,neutral
1,The Swedish buyout firm has sold its remaining...,neutral
2,$SPY wouldn't be surprised to see a green close,positive
3,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
...,...,...
4377,Investments in product development stood at 6....,neutral
4378,HSBC Says Unit to Book $585 Million Charge on ...,negative
4379,RISING costs have forced packaging producer Hu...,negative
4380,"In the building and home improvement trade , s...",neutral


In [315]:
from transformers import pipeline

model.config.num_labels >= 2
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [316]:
preds = classifier(dataset['Sentence'].tolist())

In [317]:
y_preds = [value['label'].lower() for value in preds]

In [320]:
temp = dataset.copy()
temp['Preds'] = y_preds

In [322]:
temp[temp['Sentiment'] != 'neutral']

,Sentence,Sentiment,Preds
2,SPY wouldnt surprised see green close,positive,positive
3,Shells 70 Billion BG Deal Meets Shareholder Sk...,negative,negative
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative,negative
6,Circulation revenue increased 5 Finland 4 Swed...,positive,positive
7,SAP Q1 disappoints software licenses Real prob...,negative,negative
...,...,...,...
4373,SBUX PM DB downgrade PT cut 70 64,negative,negative
4375,Finnish developer manufacturer mobile phone ch...,negative,negative
4378,HSBC Says Unit Book 585 Million Charge Settlement,negative,negative
4379,RISING costs forced packaging producer Huhtama...,negative,negative


In [323]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(temp[temp['Sentiment'] != 'neutral']['Preds'], temp[temp['Sentiment'] != 'neutral']['Sentiment'])
f1_score = f1_score(temp[temp['Sentiment'] != 'neutral']['Preds'], temp[temp['Sentiment'] != 'neutral']['Sentiment'], average='macro')

print(f'accuracy: {accuracy}, f1_score: {f1_score}')

accuracy: 0.620604259534423, f1_score: 0.6203016268641766


# Model Testing

Please report the performance of your model on the training set. How does your model perform? Please report your accuracy and F1 score. Also, using the test set, please provide a CSV of your predicted values for “Sentiment” with your submission.

# References

https://www.nasdaq.com/market-activity/stocks/screener

In [327]:
dataset

,Sentence,Sentiment
0,According Finnish-Russian Chamber Commerce maj...,neutral
1,The Swedish buyout firm sold remaining 224 per...,neutral
2,SPY wouldnt surprised see green close,positive
3,Shells 70 Billion BG Deal Meets Shareholder Sk...,negative
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
...,...,...
4377,Investments product development stood 60 mln e...,neutral
4378,HSBC Says Unit Book 585 Million Charge Settlement,negative
4379,RISING costs forced packaging producer Huhtama...,negative
4380,In building home improvement trade sales decre...,neutral


In [356]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential

In [364]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset["Sentence"])

In [366]:
X = tokenizer.texts_to_sequences(dataset["Sentence"])

# pad to same length
X = pad_sequences(X, maxlen=pd.Series(X).apply(len).max())
X.shape

(4382, 48)

In [367]:
X

array([[   0,    0,    0, ...,   17,   16,  124],
       [   0,    0,    0, ...,    3,  490,   17],
       [   0,    0,    0, ...,  347, 1032,  257],
       ...,
       [   0,    0,    0, ..., 2413,  182,   78],
       [   0,    0,    0, ...,    2, 1591,    4],
       [   0,    0,    0, ...,  158, 4941,  164]])

In [368]:
y = dataset["Sentiment"].replace(['negative', 'neutral', 'positive'],
                        [-1, 0, 1]).to_numpy()

In [369]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [370]:
max_seq_len = dataset["Sentence"].apply(len).max()
emb_dim = 100
cell_dim = 128
num_classes = 3
vocab_size = len(tokenizer.word_index) + 1
penalty = 0.01

In [371]:
l2 = keras.regularizers.l2(penalty)
model = Sequential()
model.add(Embedding(vocab_size, emb_dim))
model.add(LSTM(cell_dim, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation="softmax", kernel_regularizer=l2))

In [372]:
class saveBest(keras.callbacks.Callback):
    """Callback to save weights that maximize test accuracy"""
    def __init__(self):
        super(saveBest, self).__init__()
        self.best = {"Weights": None, "acc":float("-inf")}
        self.test_acc = []
    def on_epoch_end(self, epoch, logs=None):
        loss, acc = self.model.evaluate(x_test, y_test, verbose=False)
        if acc > self.best["acc"]:
            self.best["Weights"] = self.model.get_weights()
            self.best["acc"] = acc
        self.test_acc.append(acc)
save = saveBest()

In [373]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [374]:
y_train.shape

(3286,)

In [375]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64, callbacks=[save])

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\ericw\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ericw\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ericw\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\ericw\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\ericw\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\ericw\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\ericw\anaconda